# Import required modules

In [1]:
import os
from langchain.llms import HuggingFaceHub

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SimpleSequentialChain

from langchain.chains import SequentialChain

In [2]:
# !pip install python-dotenv

In [3]:
from dotenv import load_dotenv

load_dotenv()

True

In [4]:
HUGGINGFACEHUB_API_TOKEN = os.getenv('huggingface_api_key')

In [5]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

In [6]:
model_id = "google/flan-t5-xxl"

llm = HuggingFaceHub(repo_id=model_id, model_kwargs={"temperature":0.2, "max_length":64})

# Simple Sequential Chain

In [7]:
prompt_template_name = PromptTemplate(input_variables=["cusine"],
                                      template="I want to open a {cusine} resturant, suggest a fancy name for this"
                                      )

name_chain = LLMChain(llm=llm, prompt=prompt_template_name)

In [8]:
prompt_template_items = PromptTemplate(input_variables=["resturant_name"],
                                      template="Suggest 5 food menu items for {resturant_name} resturant."
                                      )

food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items)

In [9]:
overall_chain = SimpleSequentialChain(
    chains=[name_chain, food_items_chain]
)

response = overall_chain.run("Indian")
print(response)

The Spice Merchant has a great selection of wines and beers. It also has a great selection of appetizers and salads.


# Sequential Chain

In [10]:
llm = HuggingFaceHub(repo_id=model_id, model_kwargs={"temperature":0.2, "max_length":64})

prompt_template_name = PromptTemplate(input_variables=["cuisine"],
                                      template="I want to open a {cuisine} resturant, suggest a fancy name for this"
                                      )

name_chain = LLMChain(llm=llm, prompt=prompt_template_name, output_key="restaurant_name")


llm = HuggingFaceHub(repo_id=model_id, model_kwargs={"temperature":0.2, "max_length":64})

prompt_template_items = PromptTemplate(input_variables=["restaurant_name"],
                                      template="Suggest 5 food items which can be added in the menu for {restaurant_name}."
                                      )

food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items, output_key="menu_items")

In [11]:
chain = SequentialChain(
    chains=[name_chain, food_items_chain],
    input_variables=["cuisine"],
    output_variables=["restaurant_name", "menu_items"]
)

chain({"cuisine":"Arabic"})

{'cuisine': 'Arabic',
 'restaurant_name': 'The Aladdin',
 'menu_items': 'Chicken tikka, naan, samosa, dal makhani, naan kulfi'}